In [1]:
import tensorflow as tf
from collections import Counter
import keras
import Levenshtein
import pickle
import json
import zipfile
from tqdm.auto import tqdm
import pickle
import json
import pickle
import os
from collections import Counter
from gensim.models import Word2Vec
import os
import os
from gensim.models import Word2Vec
import numpy as np # Import numpy for vector display

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np 

2025-05-21 15:31:51.740787: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-21 15:31:52.231628: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-21 15:31:52.323301: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-05-21 15:31:52.323323: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [2]:
# --- MonographInitial.ipynb - Modified Cell 2 ---

# <<< MODIFICATION: Define how many records you want to load >>>
MAX_RECORDS_TO_LOAD = 50000

output_filepath = 'data/title_content_pair.pkl'
title_content_list = []
records_loaded = 0

# Create the 'data' directory if it doesn't exist
os.makedirs('data', exist_ok=True)

print(f"Attempting to load up to {MAX_RECORDS_TO_LOAD} records...")

with open('/home/daniel-diaz/Downloads/signalmedia-1m.jsonl/sample-1M.jsonl', 'r') as fp:
    for line in fp:
        if records_loaded >= MAX_RECORDS_TO_LOAD:
            print(f"Reached limit of {MAX_RECORDS_TO_LOAD} records.")
            break

        if line:
            try:
                data = json.loads(line.strip())
                title = data.get('title')
                content = data.get('content')
                if title and content:
                    # --- MODIFICATION: Add START and END tokens to titles HERE ---
                    processed_title = '<start> ' + title.strip() + ' <end>'
                    processed_content = content.strip() # Content doesn't need start/end

                    # Append the processed title and content
                    title_content_list.append({'title': processed_title, 'content': processed_content})
                    records_loaded += 1

            except json.JSONDecodeError as e:
                print(f"Error decoding JSON: {e} on line {records_loaded + 1}")

# Save the list of title-content dictionaries to a pickle file
with open(output_filepath, 'wb') as outfile:
    pickle.dump(title_content_list, outfile)

print(f"Successfully saved {len(title_content_list)} title-content pairs (out of a maximum of {MAX_RECORDS_TO_LOAD}) to '{output_filepath}'.")

Attempting to load up to 50000 records...
Reached limit of 50000 records.
Successfully saved 50000 title-content pairs (out of a maximum of 50000) to 'data/title_content_pair.pkl'.


In [3]:

filepath = 'data/title_content_pair.pkl'
with open(filepath, 'rb') as fp:
    loaded_data = pickle.load(fp)
title_list = []
content_list =[]

for item in loaded_data:
    title = item.get('title')
    title_list.append(title)
    
    content = item.get('content')
    content_list.append(content)
    # Now you can work with the individual title and content
    # For example, you can print them:
print(title_list[2])


<start> The Return Of The Nike Air Max Sensation Has 80’s Babies Hyped! <end>


In [4]:
title_list[5]
content_list[1]

'New Product Gives Marketers Access to Real Keywords, Conversions and Results Along With 13 Months of Historical Data \n\nSAN FRANCISCO, CA -- (Marketwired) -- 09/17/15 -- Jumpshot, a marketing analytics company that uses distinctive data sources to paint a complete picture of the online customer journey, today announced the launch of Jumpshot Elite, giving marketers insight into what their customers are doing the 99% of the time they\'re not on your site. For years, marketers have been unable to see what organic and paid search terms users were entering, much less tie those searches to purchases. Jumpshot not only injects that user search visibility back into the market, but also makes it possible to tie those keywords to conversions -- for any web site. \n\n"Ever since search engines encrypted search results, marketers have been in the dark about keywords, impacting not only the insight into their own search investments, but also their ability to unearth high converting keywords for 

In [5]:
# In Cell 5:


def get_vocab_and_counts(list_of_texts):
    # Consider adding text cleaning (lowercase, remove punctuation) here
    word_counts = Counter(word for text in list_of_texts for word in text.split())
    vocabulary_list = list(word_counts.keys()) # <<< Return a LIST
    return vocabulary_list, word_counts

all_texts = title_list + content_list
vocabulary, word_counts = get_vocab_and_counts(all_texts) # 'vocabulary' is now a list

print(f"Vocabulary size: {len(vocabulary)}")
print("Vocabulary (first 10):", vocabulary[:10])
print("Word Counts (first 10):", dict(list(word_counts.items())[:10]))

Vocabulary size: 825035
Vocabulary (first 10): ['<start>', 'Worcester', 'breakfast', 'club', 'for', 'veterans', 'gives', 'hunger', 'its', 'marching']
Word Counts (first 10): {'<start>': 50000, 'Worcester': 125, 'breakfast': 370, 'club': 1506, 'for': 194210, 'veterans': 425, 'gives': 1906, 'hunger': 201, 'its': 36517, 'marching': 93}


In [6]:

print(f"Preparing sentences for Word2Vec from {len(all_texts)} texts...")

# Tokenize the texts: Convert each text string into a list of words
# Simple tokenization: lowercase and split by space.
# You might want more sophisticated cleaning/tokenization here.
sentences = [text.lower().split() for text in all_texts]

print(f"Created {len(sentences)} tokenized sentences.")

# Optional: Check the first few tokenized sentences
print("\nExample tokenized sentences (first 3):")
for i in range(min(3, len(sentences))):
    # Print first 20 tokens, or fewer if sentence is short
    print(sentences[i][:min(20, len(sentences[i]))])
    


Preparing sentences for Word2Vec from 100000 texts...
Created 100000 tokenized sentences.

Example tokenized sentences (first 3):
['<start>', 'worcester', 'breakfast', 'club', 'for', 'veterans', 'gives', 'hunger', 'its', 'marching', 'orders', '<end>']
['<start>', 'jumpshot', 'gives', 'marketers', 'renewed', 'visibility', 'into', 'paid', 'and', 'organic', 'keywords', 'with', 'launch', 'of', 'jumpshot', 'elite', '<end>']
['<start>', 'the', 'return', 'of', 'the', 'nike', 'air', 'max', 'sensation', 'has', '80’s', 'babies', 'hyped!', '<end>']


In [7]:
START_TOKEN = '<start>'
END_TOKEN = '<end>'

In [8]:
#skip this cell
# --- In a new cell ---

# Define Word2Vec parameters
VECTOR_SIZE = 100  # Dimensionality of the word vectors (similar to EMBEDDING_DIM in GloVe)
WINDOW = 5         # Max distance between current and predicted word within a sentence
MIN_COUNT = 5      # Ignores words with total frequency lower than this
WORKERS = os.cpu_count() # Use all available CPU cores for faster training
SG = 1             # Training algorithm: 1 for skip-gram; 0 for CBOW. Skip-gram often works better.

print(f"\nTraining Word2Vec model with vector_size={VECTOR_SIZE}, window={WINDOW}, min_count={MIN_COUNT}...")

# Instantiate and train the model
# The training happens automatically when the object is created
word2vec_model = Word2Vec(sentences=sentences,
                          vector_size=VECTOR_SIZE,
                          window=WINDOW,
                          min_count=MIN_COUNT,
                          workers=WORKERS,
                          sg=SG)

print("Word2Vec model training complete.")

# --- Optional: Save the trained model for later use ---
model_save_path = "data/word2vec_model.model"
os.makedirs('data', exist_ok=True) # Ensure data directory exists
word2vec_model.save(model_save_path)
print(f"Model saved to '{model_save_path}'")

# --- To load the model later ---
# loaded_model = Word2Vec.load(model_save_path)


Training Word2Vec model with vector_size=100, window=5, min_count=5...
Word2Vec model training complete.
Model saved to 'data/word2vec_model.model'


In [9]:
#continue from here


# --- Configuration ---
model_path = "data/word2vec_model.model"
words_to_inspect = ['train'] # Words to check similarity for
num_similar = 5 # How many similar words to show for each test word
# --- ---
word2vec_model = Word2Vec.load(model_path)
# Check if the model file exists before trying to load
if not os.path.exists(model_path):
    print(f"Error: Model file not found at '{model_path}'")
    print("Please ensure the Word2Vec training cell ran successfully and saved the model.")
else:
    print(f"Loading Word2Vec model from '{model_path}'...")
    # Load the trained model
    model = Word2Vec.load(model_path)
    print("Model loaded successfully.")

    # 1. Inspect the Vocabulary
    print("\n--- Vocabulary Inspection ---")
    vocab_list = model.wv.index_to_key # List of words in the vocabulary
    vocab_size = len(vocab_list)
    print(f"Vocabulary Size: {vocab_size}")
    if vocab_size > 0:
        print(f"First 20 words in vocabulary: {vocab_list[:20]}")
    else:
        print("Vocabulary is empty.")

    # Check if the model has a vocabulary before proceeding
    if vocab_size > 0:
        # 2. Get the Vector for a Specific Word
        print("\n--- Word Vector Inspection ---")
        # Pick a word likely to be in the vocab (use the first word if possible)
        sample_word = vocab_list[0]
        try:
            vector = model.wv[sample_word]
            print(f"Vector for the word '{sample_word}':")
            print(f"  Dimensions: {model.vector_size}") # Or vector.shape[0]
            # Display only the first few dimensions for brevity
            print(f"  First 10 dimensions: {np.round(vector[:10], 4)}")
        except KeyError:
            # This should ideally not happen if we pick from vocab_list, but good practice
            print(f"Could not retrieve vector for '{sample_word}' (unexpected error).")

        # 3. Find Similar Words (Most Informative)
        print("\n--- Word Similarity Inspection ---")
        for word in words_to_inspect:
            if word in model.wv: # Check if the word exists in the model's vocabulary
                try:
                    # Get the most similar words (returns list of tuples: (word, similarity_score))
                    similar_words = model.wv.most_similar(word, topn=num_similar)
                    print(f"Words most similar to '{word}':")
                    for similar_word, score in similar_words:
                        print(f"  - {similar_word} (Score: {score:.4f})")
                except Exception as e:
                    print(f"Could not find similar words for '{word}': {e}")
            else:
                print(f"Word '{word}' is not in the model's vocabulary.")
    else:
        print("\nSkipping vector and similarity inspection because the vocabulary is empty.")
        


Loading Word2Vec model from 'data/word2vec_model.model'...
Model loaded successfully.

--- Vocabulary Inspection ---
Vocabulary Size: 131000
First 20 words in vocabulary: ['the', 'and', 'to', 'of', 'a', 'in', 'for', 'is', 'on', 'that', 'with', 'at', 'as', 'was', 'by', 'it', 'be', 'are', 'from', 'this']

--- Word Vector Inspection ---
Vector for the word 'the':
  Dimensions: 100
  First 10 dimensions: [ 0.0714 -0.2894 -0.0038  0.0561 -0.208  -0.1091 -0.1322  0.1356  0.0454
 -0.04  ]

--- Word Similarity Inspection ---
Words most similar to 'train':
  - trains (Score: 0.7759)
  - train, (Score: 0.7665)
  - carriages (Score: 0.7662)
  - keleti (Score: 0.7406)
  - london-bound (Score: 0.7381)


In [10]:
# --- MonographInitial.ipynb - MODIFIED Cell 9 ---

MAX_NUM_WORDS = 100000
OOV_TOKEN = "<OOV>"

# Define a simpler filter. By default, Tokenizer splits on spaces and punctuation in the filters string.
# Removing most punctuation from filters means it will only split on tabs and newlines,
# preserving tokens that contain characters like <, >, or other symbols.
TOKENIZER_FILTERS = '\t\n' # Minimal filter: only remove tabs and newlines

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, oov_token=OOV_TOKEN, filters=TOKENIZER_FILTERS)

print(f"Tokenizer initialized with filters: '{TOKENIZER_FILTERS}'")
print(f"MAX_NUM_WORDS: {MAX_NUM_WORDS}, OOV_TOKEN: '{OOV_TOKEN}'")

# --- End of MODIFIED Cell 9 ---

Tokenizer initialized with filters: '	
'
MAX_NUM_WORDS: 100000, OOV_TOKEN: '<OOV>'


In [11]:
# --- MonographInitial.ipynb - CORRECTED Cell 10 ---

# Ensure necessary variables are defined from previous cells or define them here for clarity
# title_list, content_list must come from Cell 3 (which got them from the pickle saved by Cell 2)
# MAX_NUM_WORDS, OOV_TOKEN must come from Cell 9
# TOKENIZER_FILTERS must come from Cell 9
# START_TOKEN, END_TOKEN must come from Cell 13 or define them here

# Define tokens and key parameters again for robustness in this cell
START_TOKEN = '<start>'
END_TOKEN = '<end>'
# Assuming MAX_NUM_WORDS, OOV_TOKEN, TOKENIZER_FILTERS are defined in Cell 9


print("Executing Cell 10: Tokenizer fitting and saving...")

# --- Explicitly recreate all_texts RIGHT NOW ---
# This ensures we are using the most recent title_list and content_list
if 'title_list' in globals() and 'content_list' in globals():
    all_texts_for_tokenizer = title_list + content_list
    print(f"Recreated all_texts for tokenizer fitting. Size: {len(all_texts_for_tokenizer)}")
    # Optional: Verify first few items contain tokens
    if len(all_texts_for_tokenizer) > 0:
        print(f"First text in all_texts for tokenizer: {all_texts_for_tokenizer[0]}")
        print(f"Second text in all_texts for tokenizer: {all_texts_for_tokenizer[1]}")
else:
    print("Error: title_list or content_list not found. Cannot create all_texts for tokenizer.")
    # You might want to stop execution here if the lists aren't found
    # exit() # You can uncomment this to stop the notebook execution if data is missing


# --- Initialize tokenizer with the custom filters ---
# Ensure MAX_NUM_WORDS, OOV_TOKEN, and TOKENIZER_FILTERS are defined (should be from Cell 9)
if 'MAX_NUM_WORDS' not in globals() or 'OOV_TOKEN' not in globals() or 'TOKENIZER_FILTERS' not in globals():
     print("Error: MAX_NUM_WORDS, OOV_TOKEN, or TOKENIZER_FILTERS not defined (from Cell 9). Cannot initialize tokenizer.")
     # exit() # You can uncomment this to stop the notebook execution if parameters are missing

# This is the corrected line: Include the filters argument
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, oov_token=OOV_TOKEN, filters=TOKENIZER_FILTERS)
print(f"Tokenizer initialized with custom filters: '{TOKENIZER_FILTERS}'") # Added print detail

# --- Fit the tokenizer using the explicitly recreated all_texts ---
print("Fitting tokenizer on all_texts...")
# Check if all_texts_for_tokenizer was successfully created
if 'all_texts_for_tokenizer' in globals() and all_texts_for_tokenizer:
    tokenizer.fit_on_texts(all_texts_for_tokenizer)
    print("Tokenizer fitting complete.")
else:
    print("Error: all_texts_for_tokenizer is not available or empty. Cannot fit tokenizer.")
    # exit() # You can uncomment this to stop if fitting failed


# --- Verification of Tokenizer Immediately After Fit ---
# These checks need START_TOKEN and END_TOKEN defined (from definitions above or Cell 13)
if 'START_TOKEN' in globals() and 'END_TOKEN' in globals() and 'tokenizer' in globals():
     print("\n--- Verification of Tokenizer after Fit ---")
     print(f"Tokenizer fitted vocabulary size: {len(tokenizer.word_index)}")
     # Note: Check in tokenizer.word_index, which contains all words found before filtering by num_words
     contains_start = START_TOKEN in tokenizer.word_index
     contains_end = END_TOKEN in tokenizer.word_index
     print(f"Does tokenizer vocabulary contain '{START_TOKEN}'? {contains_start}")
     print(f"Does tokenizer vocabulary contain '{END_TOKEN}'? {contains_end}")

     start_id_check = tokenizer.word_index.get(START_TOKEN, None) # Use None default here
     end_id_check = tokenizer.word_index.get(END_TOKEN, None) # Use None default here
     print(f"ID for '{START_TOKEN}': {start_id_check}")
     print(f"ID for '{END_TOKEN}': {end_id_check}")

     # --- Save the tokenizer ONLY IF verification passes ---
     # It's also good to check if the IDs are within the VOCAB_SIZE range IF you need them there
     # But for this specific check, just verifying they exist and have IDs is enough.
     if contains_start and contains_end and start_id_check is not None and end_id_check is not None:
          print("\nVerification successful. Proceeding to save tokenizer.")
          tokenizer_path = 'data/tokenizer.pkl' # Define the path
          os.makedirs('data', exist_ok=True) # Ensure the 'data' directory exists
          with open(tokenizer_path, 'wb') as handle:
              pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
          print(f"Tokenizer successfully saved to '{tokenizer_path}'")
     else:
          print("\nVerification failed. Tokenizer does NOT contain START/END tokens (or IDs are None). Not saving tokenizer.")
          print("This usually means the data loading (Cell 2) didn't include the tokens correctly, or the filters (Cell 9) were not applied/set up correctly.") # More detailed error message

     print("--- End Verification/Save Process ---")

else:
     # This else is for the initial check if START_TOKEN/END_TOKEN/tokenizer are defined before verification block
     print("\nWarning: Tokenizer or necessary tokens (START_TOKEN, END_TOKEN) not defined before verification. Skipping tokenizer verification and saving.")
     print("Please ensure Cell 9 and preceding cells defining tokens are run.")


# --- End of CORRECTED Cell 10 ---

Executing Cell 10: Tokenizer fitting and saving...
Recreated all_texts for tokenizer fitting. Size: 100000
First text in all_texts for tokenizer: <start> Worcester breakfast club for veterans gives hunger its marching orders <end>
Second text in all_texts for tokenizer: <start> Jumpshot Gives Marketers Renewed Visibility Into Paid and Organic Keywords With Launch of Jumpshot Elite <end>
Tokenizer initialized with custom filters: '	
'
Fitting tokenizer on all_texts...
Tokenizer fitting complete.

--- Verification of Tokenizer after Fit ---
Tokenizer fitted vocabulary size: 786177
Does tokenizer vocabulary contain '<start>'? True
Does tokenizer vocabulary contain '<end>'? True
ID for '<start>': 36
ID for '<end>': 37

Verification successful. Proceeding to save tokenizer.
Tokenizer successfully saved to 'data/tokenizer.pkl'
--- End Verification/Save Process ---


In [12]:
VOCAB_SIZE = MAX_NUM_WORDS 

In [13]:
print(f"Tokenizer created. Vocabulary size: {VOCAB_SIZE}")

Tokenizer created. Vocabulary size: 100000


In [14]:
START_TOKEN = '<start>'
END_TOKEN = '<end>'

In [15]:
# --- Add this as a NEW cell AFTER Cell 13 in MonographInitial.ipynb ---

print("\n--- Verification of Tokenizer after Definitions ---")
# Check if tokenizer and necessary tokens are defined BEFORE using them in checks
if 'tokenizer' in globals() and 'START_TOKEN' in globals() and 'END_TOKEN' in globals():
     print(f"Tokenizer fitted vocabulary size: {len(tokenizer.word_index)}")
     print(f"Does tokenizer vocabulary contain '{START_TOKEN}'? {START_TOKEN in tokenizer.word_index}")
     print(f"Does tokenizer vocabulary contain '{END_TOKEN}'? {END_TOKEN in tokenizer.word_index}")
     start_id_check = tokenizer.word_index.get(START_TOKEN)
     end_id_check = tokenizer.word_index.get(END_TOKEN)
     print(f"ID for '{START_TOKEN}': {start_id_check}")
     print(f"ID for '{END_TOKEN}': {end_id_check}")
else:
     print("Warning: Tokenizer or necessary tokens not defined. Cannot perform verification.")

print("--- End Verification ---")

# --- End of new cell ---


--- Verification of Tokenizer after Definitions ---
Tokenizer fitted vocabulary size: 786177
Does tokenizer vocabulary contain '<start>'? True
Does tokenizer vocabulary contain '<end>'? True
ID for '<start>': 36
ID for '<end>': 37
--- End Verification ---


In [16]:
EMBEDDING_DIM = word2vec_model.vector_size

In [17]:
embedding_matrix = np.zeros((VOCAB_SIZE, EMBEDDING_DIM))

# Fill the matrix with Word2Vec vectors
# Start from index 1 because index 0 is for padding
for word, i in tokenizer.word_index.items():
    if i < VOCAB_SIZE: # Ensure we stay within our desired vocabulary size
        # Check if the word exists in the Word2Vec model's vocabulary
        if word in word2vec_model.wv:
            embedding_matrix[i] = word2vec_model.wv[word]
        else:
            # Optional: Handle words not found in Word2Vec (e.g., leave as zeros, or random init)
            # print(f"Warning: Word '{word}' not found in Word2Vec vocabulary. Initializing with zeros.")
            pass # Keep as zero if initialized with zeros

In [18]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense

In [19]:
MAX_ARTICLE_LEN =500 # Example length
MAX_HEADLINE_LEN = 50 # Example length

In [20]:
encoder_inputs = Input(shape=(MAX_ARTICLE_LEN,), name='encoder_input')


RNN_SIZE = 256


DROPOUT_RATE = 0.2 

print(f"RNN Size set to: {RNN_SIZE}")
print(f"Dropout Rate set to: {DROPOUT_RATE}")

RNN Size set to: 256
Dropout Rate set to: 0.2


In [21]:
# --- In Cell 23 ---

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense

# Keep your MAX_ARTICLE_LEN and MAX_HEADLINE_LEN definitions as they are
MAX_ARTICLE_LEN =500 # Example length
MAX_HEADLINE_LEN = 50 # Example length

encoder_inputs = Input(shape=(MAX_ARTICLE_LEN,), name='encoder_input')

# Keep your RNN_SIZE and DROPOUT_RATE definitions as they are
RNN_SIZE = 256
DROPOUT_RATE = 0.2

print(f"RNN Size set to: {RNN_SIZE}")
print(f"Dropout Rate set to: {DROPOUT_RATE}")

# Keep this as it is (encoder embedding)
encoder_embedding = Embedding( input_dim=VOCAB_SIZE,
                              output_dim=EMBEDDING_DIM,
                              weights=[embedding_matrix],
                              trainable=True,
                              mask_zero=True,
                              name='shared_embedding0')(encoder_inputs)

# Keep encoder LSTMs as they are (they already have return_state=True)
encoder_lstm1 = LSTM(RNN_SIZE, return_sequences=True, return_state=True, name='encoder_lstm_1')(encoder_embedding)
encoder_lstm2 = LSTM(RNN_SIZE, return_sequences=True, return_state=True, name='encoder_lstm_2')(encoder_lstm1[0])
encoder_lstm3 = LSTM(RNN_SIZE, return_sequences=True, return_state=True, name='encoder_lstm_3')(encoder_lstm2[0])

# Keep encoder states as they are (take states from the *last* LSTM)
encoder_states = [encoder_lstm3[1], encoder_lstm3[2]]

# Keep decoder_inputs as they are
decoder_inputs = Input(shape=(MAX_HEADLINE_LEN,), name='decoder_input')

# Keep decoder embedding as it is (ensure name matches inference lookup: 'shared_embedding1')
decoder_embedding = Embedding(input_dim=VOCAB_SIZE,
                              output_dim=EMBEDDING_DIM,
                              weights=[embedding_matrix],
                              trainable=True,
                              mask_zero=True,
                              name='shared_embedding1')(decoder_inputs)

# THIS WAS THE LINE TO CHANGE - YOU NEED TO SELECT THE SEQUENCE OUTPUT [0]
decoder_lstm1 = LSTM(RNN_SIZE,
                     return_sequences=True,
                     return_state=True,
                     name='decoder_lstm_1')(decoder_embedding, initial_state=encoder_states)

# Corrected line: Pass only the sequence output [0] to the Dense layer
decoder_dense = Dense(VOCAB_SIZE, activation='softmax', name='output_layer')(decoder_lstm1[0]) # <--- FIXED THIS LINE

model = Model([encoder_inputs, decoder_inputs], decoder_dense, name='seq2seq_model')

print("Seq2Seq model structure defined.")
model.summary() # Look at the summary again to confirm shapes

RNN Size set to: 256
Dropout Rate set to: 0.2


2025-05-21 15:35:34.184594: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2025-05-21 15:35:34.184618: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2025-05-21 15:35:34.184633: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (daniel-diaz-Latitude-3520): /proc/driver/nvidia/version does not exist
2025-05-21 15:35:34.186645: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Seq2Seq model structure defined.
Model: "seq2seq_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input (InputLayer)     [(None, 500)]        0           []                               
                                                                                                  
 shared_embedding0 (Embedding)  (None, 500, 100)     10000000    ['encoder_input[0][0]']          
                                                                                                  
 encoder_lstm_1 (LSTM)          [(None, 500, 256),   365568      ['shared_embedding0[0][0]']      
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                     

In [22]:
encoder_input_sequences = tokenizer.texts_to_sequences(content_list)
decoder_input_sequences = tokenizer.texts_to_sequences(title_list)
decoder_target_sequences = tokenizer.texts_to_sequences(title_list)
encoder_input_sequences = pad_sequences(encoder_input_sequences, maxlen=MAX_ARTICLE_LEN, padding='post')
decoder_input_sequences = pad_sequences(decoder_input_sequences, maxlen=MAX_HEADLINE_LEN, padding='post')
decoder_target_sequences = pad_sequences(decoder_target_sequences, maxlen=MAX_HEADLINE_LEN, padding='post')
START_TOKEN_ID = tokenizer.word_index.get(START_TOKEN, 1)
END_TOKEN_ID = tokenizer.word_index.get(END_TOKEN, 2)

In [23]:
# --- NEW Cell: Load Model from Checkpoint to Resume Training ---

import tensorflow as tf
import os

# Define the path to the checkpoint file you want to resume from
# Make sure this path is correct!
resume_checkpoint_path = 'training_checkpoints/epoch_50_val_loss_1.4797.h5' # <--- Verify this path

if not os.path.exists(resume_checkpoint_path):
    print(f"Error: Checkpoint file not found at '{resume_checkpoint_path}'")
    print("Cannot resume training. Please check the path or train from scratch.")
    # You might want to stop execution here if the checkpoint is essential
else:
    print(f"Loading model from checkpoint: '{resume_checkpoint_path}' to resume training...")

    # Load the entire model, including architecture, weights, and optimizer state
    # Keras automatically preserves the compilation state when loading the full model
    model = tf.keras.models.load_model(resume_checkpoint_path)

    print("Model loaded successfully. Ready to resume training.")

    # Optional: Print current optimizer state if you want to inspect
    # print(f"Optimizer state loaded: {model.optimizer.get_config()}")

# --- End of NEW Cell ---

Error: Checkpoint file not found at 'training_checkpoints/epoch_50_val_loss_1.4797.h5'
Cannot resume training. Please check the path or train from scratch.


In [ ]:
# --- MonographInitial.ipynb - Cell 24 (COMPLETE code for Resuming Training with Checkpointing and Plotting) ---

# Ensure you have matplotlib imported at the top of your notebook (e.g., in Cell 1)
import matplotlib.pyplot as plt # Added import for plotting here for clarity, but should be at the top
import os # Needed for directory creation and path joining

# Make sure encoder_input_sequences, decoder_input_sequences,
# decoder_target_sequences are defined (should be from Cell 22).
# Make sure MAX_ARTICLE_LEN, MAX_HEADLINE_LEN, BATCH_SIZE are defined (from earlier cells).
# Make sure the 'model' variable is populated from Cell 23 (the loading cell).

BATCH_SIZE = 100
# --- Variable Definitions for Resuming/Training ---
# If you want to train for an additional number of epochs:
NEW_TOTAL_EPOCHS = 100 # Set your desired total number of epochs here (e.g., 50 + 50 = 100)

# The epoch to start counting from (the one AFTER the loaded checkpoint's epoch number)
# If you loaded epoch_50, you start from epoch 51 for the progress counter and callbacks
STARTING_EPOCH_NUMBER = 50 # <--- Set this to the epoch number you just loaded FROM (50 in your case)


# Ensure model is defined (loaded in Cell 23)
if 'model' not in globals() or model is None:
    print("Error: Model variable 'model' is not defined or is None. Please run Cell 23 to load the model.")
    # You might want to stop execution here if the model is essential
    # exit() # Uncomment this line if you want to stop automatically


# --- 1. Setup Model Checkpointing ---

# Define the directory where checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Create the directory if it doesn't exist
os.makedirs(checkpoint_dir, exist_ok=True)

# Define the filename format for checkpoints
# Includes epoch number ({epoch:02d}) and validation loss ({val_loss:.4f})
# The {epoch} placeholder will use the initial_epoch value correctly for numbering
checkpoint_filepath = os.path.join(checkpoint_dir, 'epoch_{epoch:02d}_val_loss_{val_loss:.4f}.h5')

# Create the ModelCheckpoint callback instance
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False, # Set to True if you only want to save weights, False saves entire model
    monitor='val_loss',      # Metric to monitor (included in filename)
    mode='min',              # Mode for the monitored metric
    save_best_only=False,    # <--- IMPORTANT: Set to False to save AFTER *every* epoch
    save_freq='epoch',       # <--- IMPORTANT: Set to 'epoch' to trigger saving at the end of each epoch
    verbose=1                # Print messages when saving a checkpoint
)

print("ModelCheckpoint callback setup complete.")
print(f"Checkpoints will be saved to: {checkpoint_dir}")
print(f"Filename pattern: {os.path.basename(checkpoint_filepath)}")


# --- 2. Compile the model (if necessary - loading full model often preserves compile state) ---
# You generally don't *need* to re-compile if loading the full model using tf.keras.models.load_model,
# as the compilation state (optimizer, loss, metrics) is saved.
# However, calling compile again doesn't hurt if the arguments are the same, and is necessary
# if you changed compile args or if you loaded weights only.
# Let's keep it here but note it might be redundant after loading the full model.
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
print("Model compiled (or confirmed compilation state).")


# --- 3. Start training with the callback ---
print(f"Resuming model training from epoch {STARTING_EPOCH_NUMBER + 1} up to epoch {NEW_TOTAL_EPOCHS}...")

history = model.fit([encoder_input_sequences, decoder_input_sequences],
                    decoder_target_sequences,
                    epochs=NEW_TOTAL_EPOCHS,        # <--- Train up to this total epoch number
                    initial_epoch=STARTING_EPOCH_NUMBER, # <--- Tell Keras to start counting epochs from here
                    batch_size=BATCH_SIZE,
                    validation_split=0.2,
                    callbacks=[model_checkpoint_callback]) # <--- Pass the callback list here

print("Training finished.")


# --- 4. Plot Loss History ---
# The history object from fit will contain data from the start of the *resumed* training.
# Plotting will show the loss from epoch STARTING_EPOCH_NUMBER+1 onwards.
print("Plotting loss history for resumed training...")

# Get the loss values from the history object
loss = history.history['loss']
# Check if validation loss was recorded (it should be with validation_split=0.2)
val_loss_available = 'val_loss' in history.history
if val_loss_available:
    val_loss = history.history['val_loss']

# Create plot
plt.figure(figsize=(12, 6))
# Adjust x-axis to start from the correct epoch number for the plot
# The range should go from STARTING_EPOCH_NUMBER + 1 up to NEW_TOTAL_EPOCHS inclusive of epochs where data was recorded
# The length of history.history['loss'] corresponds to the number of *new* epochs trained.
# So if you trained 50 original + 50 new, the length is 50. The epoch numbers are STARTING_EPOCH_NUMBER + 1, ..., STARTING_EPOCH_NUMBER + len(loss)
epochs_trained_in_this_run = len(loss)
epochs_range_for_plot = range(STARTING_EPOCH_NUMBER + 1, STARTING_EPOCH_NUMBER + epochs_trained_in_this_run + 1)


plt.plot(epochs_range_for_plot, loss, label='Training Loss')
if val_loss_available:
    plt.plot(epochs_range_for_plot, val_loss, label='Validation Loss')

plt.title('Training and Validation Loss per Epoch (Resumed)') # Updated title
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()
print("Loss plot displayed.")


# --- 5. Optional: Save the final model explicitly ---
# This is in addition to the epoch-specific checkpoints saved by the callback.
# You can keep this commented out if you prefer relying on the checkpoint files.
# If uncommented, this saves the state *after* the last epoch of the resumed training.
# print("Saving final model...")
# final_model_save_path = 'seq2seq_headline_summarizer_model_final.h5'
# # Ensure the data directory exists (it should from earlier cells, but good practice)
# os.makedirs('data', exist_ok=True) # Assuming you want the final save in data dir
# model.save(final_model_save_path) # Or specify a path like 'data/seq2seq_headline_summarizer_model_final.h5'
# print(f"Final model saved to '{final_model_save_path}'")


# --- End of MonographInitial.ipynb - Cell 24 COMPLETE ---

ModelCheckpoint callback setup complete.
Checkpoints will be saved to: ./training_checkpoints
Filename pattern: epoch_{epoch:02d}_val_loss_{val_loss:.4f}.h5
Model compiled (or confirmed compilation state).
Resuming model training from epoch 51 up to epoch 100...
Epoch 51/100
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
400/400 [==============================] - ETA: 0s - loss: 7.2058  WARNING:tensorflow:AutoGraph could 

In [ ]:
# --- START: Code for a NEW session ---
import tensorflow as tf
import numpy as np
import pickle
import os
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# --- Define the same constants as training ---
# Make sure these match the values used during training
MAX_RECORDS_TO_LOAD = 100000 # Used for data loading initially, needed for context
MAX_ARTICLE_LEN = 500       # Must match training
MAX_HEADLINE_LEN = 50       # Must match training
MAX_NUM_WORDS = 9999      # Must match tokenizer setup
OOV_TOKEN = "<OOV>"         # Must match tokenizer setup
START_TOKEN = "<start>"     # Must match tokenizer setup and data prep
END_TOKEN = "<end>"         # Must match tokenizer setup and data prep
RNN_SIZE = 256              # Must match training
# Note: EMBEDDING_DIM comes from the loaded model's embedding layer

# --- Load or Recreate the Tokenizer ---
# Ideally, save the tokenizer after fitting it in the original notebook
tokenizer_path = 'data/tokenizer.pkl' # Recommended: Save your tokenizer!

# --- Example: Save the tokenizer after fitting (run this ONCE in the original notebook) ---
# In the original notebook, after running cell 10:
# with open(tokenizer_path, 'wb') as handle:
#     pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
# ----------------------------------------------------------------------------------------

# --- Load the tokenizer in the new session ---
if os.path.exists(tokenizer_path):
    print(f"Loading tokenizer from '{tokenizer_path}'...")
    with open(tokenizer_path, 'rb') as handle:
        tokenizer = pickle.load(handle)
    print("Tokenizer loaded.")
else:
    # If tokenizer was not saved, you need to recreate it by fitting on the same data
    # This requires loading the data again or having the 'all_texts' list available.
    # Loading the saved tokenizer is much more reliable.
    print(f"Error: Tokenizer file not found at '{tokenizer_path}'.")
    print("Please save the tokenizer after fitting it in the original notebook.")
    # Fallback (less reliable if data/preprocessing changes):
    # You'd need to load 'title_content_pair.pkl' and rebuild all_texts here
    # all_texts = ... # Load data and combine titles/content
    # tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, oov_token=OOV_TOKEN)
    # tokenizer.fit_on_texts(all_texts)
    # print("Tokenizer recreated from data (less reliable than loading).")
    # VOCAB_SIZE = min(tokenizer.num_words, len(tokenizer.word_index) + 1) # +1 for OOV/padding if not included in num_words
    exit() # Stop if tokenizer loading fails

# Ensure VOCAB_SIZE is consistent with the tokenizer and training setup
# If num_words was set, VOCAB_SIZE is usually num_words.
# If num_words was None, it's len(tokenizer.word_index) + 1 (for OOV) or +2 (OOV and padding 0)
# Given you set MAX_NUM_WORDS=9000 and OOV, VOCAB_SIZE should be 9000.
VOCAB_SIZE = MAX_NUM_WORDS # Or min(MAX_NUM_WORDS, len(tokenizer.word_index) + 1)

# Get token IDs (ensure they match the tokenizer's mapping)
START_TOKEN_ID = tokenizer.word_index.get(START_TOKEN, 1) # get returns default if not found
END_TOKEN_ID = tokenizer.word_index.get(END_TOKEN, 2)   # These might be 1, 2 depending on tokenizer fit order

print(f"Tokenizer vocabulary size (MAX_NUM_WORDS): {VOCAB_SIZE}")
print(f"Start Token ID: {START_TOKEN_ID}")
print(f"End Token ID: {END_TOKEN_ID}")

In [ ]:
# --- In a new cell, after training ---

# Assuming you have the trained `model`, `tokenizer`, `MAX_ARTICLE_LEN`, `MAX_HEADLINE_LEN`,
# START_TOKEN_ID, END_TOKEN_ID, VOCAB_SIZE (from the previous steps)
# Note: Make sure START_TOKEN and END_TOKEN are added to your titles *before* tokenization
# and that VOCAB_SIZE is correctly set to MAX_NUM_WORDS as discussed in the previous answer.

print("Setting up inference models (reusing trained layers)...")

# Get the layers from the trained model
# Use the exact names from the traceback: 'shared_embedding0', 'shared_embedding1', etc.
encoder_embedding_layer = model.get_layer('shared_embedding0')
encoder_lstm1_layer = model.get_layer('encoder_lstm_1')
encoder_lstm2_layer = model.get_layer('encoder_lstm_2')
encoder_lstm3_layer = model.get_layer('encoder_lstm_3')

decoder_embedding_layer = model.get_layer('shared_embedding1')
decoder_lstm_layer = model.get_layer('decoder_lstm_1') # This is the trained decoder LSTM layer
decoder_dense_layer = model.get_layer('output_layer')

# --- Encoder Inference Model ---
# Input: Define a new Input tensor for the encoder inference model
encoder_inputs_inf = Input(shape=(MAX_ARTICLE_LEN,), name='encoder_input_inf')

# Chain the layers using the *trained* layer instances
encoder_embedding_out = encoder_embedding_layer(encoder_inputs_inf)

# Pass through the encoder LSTMs to get the final states
# Since the original model had multiple stacked LSTMs and output the last one's states,
# we need to propagate the outputs through the layers.
# The outputs of LSTMs (when return_sequences=True) are the sequences, used as input to the next LSTM.
# The return_state=True on the original LSTMs allows us to access their states.
encoder_lstm1_output, state_h1_enc_inf, state_c1_enc_inf = encoder_lstm1_layer(encoder_embedding_out)
encoder_lstm2_output, state_h2_enc_inf, state_c2_enc_inf = encoder_lstm2_layer(encoder_lstm1_output)
encoder_lstm3_output, state_h3_enc_inf, state_c3_enc_inf = encoder_lstm3_layer(encoder_lstm2_output)

# The encoder inference model outputs the states of the *last* encoder LSTM
encoder_states_inf = [state_h3_enc_inf, state_c3_enc_inf]
encoder_model = Model(encoder_inputs_inf, encoder_states_inf)

print("Encoder Inference Model set up.")


# --- Decoder Inference Model ---
# This model takes the previously predicted word and the decoder states,
# and outputs the next word probabilities and the new decoder states.

# Inputs for the decoder inference model
# 1. Input for the single word predicted at the previous step (or START token initially)
decoder_single_word_input_inf = Input(shape=(1,), name='decoder_single_word_input_inf')

# 2. Inputs for the previous decoder states (output from the encoder or previous decoder step)
decoder_state_input_h_inf = Input(shape=(RNN_SIZE,), name='decoder_state_input_h_inf')
decoder_state_input_c_inf = Input(shape=(RNN_SIZE,), name='decoder_state_input_c_inf')
decoder_states_inputs_inf = [decoder_state_input_h_inf, decoder_state_input_c_inf] # List of state inputs

# Process the single word input: Embedding layer
decoder_single_word_embedding_inf = decoder_embedding_layer(decoder_single_word_input_inf) # Use the trained embedding layer

# Pass through the *trained* decoder LSTM layer
# Provide the previous states as `initial_state`.
# Since the input shape is (None, 1, EMBEDDING_DIM), and `return_sequences=True` on the layer instance,
# the output sequence will have shape (None, 1, RNN_SIZE).
# The layer also returns the new states because `return_state=True` was on the original layer.
decoder_outputs_inf, state_h_dec_inf, state_c_dec_inf = decoder_lstm_layer(
    decoder_single_word_embedding_inf, initial_state=decoder_states_inputs_inf
)

# Apply the *trained* dense output layer
# This layer was applied to the output sequence in the training model.
# When applied to the (None, 1, RNN_SIZE) output from the inference LSTM step,
# it will produce an output shape of (None, 1, VOCAB_SIZE).
decoder_pred_output_sequence = decoder_dense_layer(decoder_outputs_inf)

# For step-by-step inference, we need the prediction for the *current* timestep.
# Since the sequence length is 1, we take the first (and only) element of the output sequence.
decoder_pred_output_inf = decoder_pred_output_sequence[:, 0, :] # Slice to get shape (None, VOCAB_SIZE)


# The decoder inference model outputs the word probabilities for the current step
# and the new decoder states for the next step.
decoder_model = Model(
    [decoder_single_word_input_inf] + decoder_states_inputs_inf,
    [decoder_pred_output_inf] + [state_h_dec_inf, state_c_dec_inf] # Output the predicted probs and new states
)

print("Decoder Inference Model set up.")
print("Inference models built by reusing trained layers. No manual weight transfer needed.")

# Note: You don't need explicit build() calls or set_weights() here
# because you are using layers that were already built and have weights
# from the original `model` during training.

